In [ ]:
# Create a Downloads directory if it doesn't exist.
!mkdir -p /content/model

# Download the model tarball from Kaggle.
!curl -L -o /content/model/model.tar.gz "https://www.kaggle.com/api/v1/models/spsayakpaul/deit/tensorFlow2/base-distilled-patch16-224/1/download"

# Extract the model tar.gz into a directory (e.g., ~/Downloads/model)
!tar -xzvf /content/model/model.tar.gz -C /content/model

In [ ]:
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow.keras as keras

MODEL_PATH = "/content/model"
res = 224
num_classes = 7


inputs = keras.Input((res, res, 3))
hub_module = hub.KerasLayer(MODEL_PATH, trainable=False)
# Explicitly build the layer to force shape resolution.
hub_module.build((None, res, res, 3))
x = hub_module(inputs, training=True)
# If the module returns a tuple, take the first element.
if isinstance(x, (list, tuple)):
    x = x[0]

# Add a new Dense layer for classification.
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024, activation="relu")(x)
x = keras.layers.Dropout(0.25)(x)
outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

# Compile the model.
model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy(), metrics=["accuracy"])

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(filepath='Swin_pretrained.keras', monitor='val_loss', save_best_only=True)
]
# Train the model.
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15,
    callbacks=callbacks
)
